In [13]:
import tensorflow as tf
import numpy as np
from tdc.single_pred import Tox
from rdkit import Chem

In [14]:
from rdkit.Chem import MolFromSmiles, MolToSmiles
from tqdm import tqdm
from SmilesPE import tokenizer
def encode(smiles_list,labels):
    corpus = list(zip(smiles_list,labels))
    mols = []
    labels = []
    len_list = []
    res_labels = []
    for index,smiles in enumerate(corpus):
        try:
            mol = MolFromSmiles(smiles[0])
        except:
            continue
        if mol == None:
            continue
        mols.append(mol)
        labels.append(smiles[1])
        
    
    print(len(mols),len(labels))
    
    corpus = list(zip(mols,labels))
    print(mols[0])
    res = []
    for index1,mol in (enumerate(tqdm(corpus))):
        index = 0
        res_temp = []
        while(len(res_temp)<20 and index <100):
            try:
                random_smiles = MolToSmiles(mol[0], doRandom=True) # e.g 'C(C(=O)O)N' 
            except:
                print(mol[0])
                raise
            res_temp.append(random_smiles)
            res_temp = list(set(res_temp))
            index+=1
        res += res_temp
        res_labels += [mol[1]]*len(res_temp)
        len_list.append(len(res_temp))
        
    # mapping atomID into SMILES srting
    res_train = []


    for smiles in tqdm(res):
        smile_tokens = tokenizer.atomwise_tokenizer(smiles)
        res_train.append(smile_tokens)

    return res_train,res_labels,len_list

In [3]:
Tox_list = ['hERG_Karim','LD50_Zhu','hERG','AMES','DILI','Skin Reaction','Carcinogens_Lagunin','ClinTox']

In [4]:
tox_train,tox_label = Tox(name='herg').get_data(format='DeepPurpose')

Found local copy...
Loading...
Done!


In [15]:
def create_variable(var_name):
    globals()[var_name] = []
def get_variable_value(var_name):
    return globals().get(var_name)


In [16]:
import pickle
for tox in Tox_list:
    create_variable(tox)
    tox_train,tox_label = Tox(name=tox).get_data(format='DeepPurpose')
    temp_train = get_variable_value(tox)
    res = encode(tox_train,tox_label)
    temp_train.append(res)
    with open(f'./Final_data_token_SMILE/TOX/{tox}','wb') as file:
        pickle.dump(temp_train,file)


Found local copy...
Loading...
Done!


13445 13445


100%|██████████| 268795/268795 [00:02<00:00, 89779.21it/s] 
Found local copy...
Loading...
Done!


7385 7385


100%|██████████| 141235/141235 [00:01<00:00, 138484.63it/s]
Found local copy...
Loading...
Done!
[01:45:09] WARNING: not removing hydrogen atom without neighbors
[01:45:09] WARNING: not removing hydrogen atom without neighbors


...

100%|██████████| 7711/7711 [00:00<00:00, 185300.09it/s]
Found local copy...
Loading...
Done!


280 280


100%|██████████| 5501/5501 [00:00<00:00, 114447.61it/s]
Found local copy...
Loading...
Done!


1478 1478


100%|██████████| 29009/29009 [00:00<00:00, 109415.90it/s]


In [18]:
from tdc.utils import retrieve_label_name_list
import pickle
Tox_list_label = ['herg_central','Tox21','ToxCast']
for tox in Tox_list_label:
    label_list = retrieve_label_name_list(tox)
    for label in label_list:

        create_variable(tox+'_'+label)
        tox_train,tox_label = Tox(name=tox,label_name=label).get_data(format='DeepPurpose')
        temp_train = get_variable_value(tox+'_'+label)
        temp_train.append(encode(tox_train,tox_label,False))
        with open('./Tox_data/SMILE_Tox_data/'+tox+'_'+label+'_can','wb') as file:
            pickle.dump(temp_train,file)

Found local copy...
Loading...
Done!
100%|██████████| 306893/306893 [00:03<00:00, 85343.30it/s] 
Found local copy...
Loading...
Done!
100%|██████████| 306893/306893 [00:02<00:00, 112148.06it/s]
Found local copy...
Loading...
Done!
100%|██████████| 306893/306893 [00:03<00:00, 98101.36it/s] 
Found local copy...
Loading...
Done!
100%|██████████| 7265/7265 [00:00<00:00, 156802.66it/s]
Found local copy...
Loading...
Done!
100%|██████████| 6758/6758 [00:00<00:00, 166782.03it/s]
Found local copy...
Loading...
Done!
100%|██████████| 6549/6549 [00:00<00:00, 157448.68it/s]
Found local copy...
Loading...
Done!
100%|██████████| 5821/5821 [00:00<00:00, 161851.95it/s]
Found local copy...
Loading...
Done!
100%|██████████| 6193/6193 [00:00<00:00, 163540.65it/s]
Found local copy...
Loading...
Done!
100%|██████████| 6955/6955 [00:00<00:00, 164210.76it/s]
Found local copy...
Loading...
Done!
100%|██████████| 6450/6450 [00:00<00:00, 171780.91it/s]
Found local copy...
Loading...
Done!
100%|██████████| 5832

In [9]:
import pandas as pd
import glob
import pickle
# CSV 파일이 있는 폴더 경로 설정
folder_path = './Final_data/CLS/'

# 폴더 내의 모든 CSV 파일 경로 가져오기
csv_files = glob.glob(folder_path + '*.csv')
tox_names = []
for file in csv_files:
    name = file.split('/')[3]
    tox_names.append(name[:-4])

# 모든 CSV 파일을 담을 빈 데이터프레임 생성
all_data = pd.DataFrame()
train_Index = ['Smiles','smiles','SMILES']
label_Index = ['Property','expt','exp','p_np']
# 각 CSV 파일을 읽어와서 데이터프레임에 추가
for index,file_path in enumerate(csv_files):
    df = pd.read_csv(file_path)
    for t in train_Index:
        try:
            train_data=df[t]
        except:
            pass
    for l in label_Index:
        try:
            label_data = df[l]
        except:
            pass
    train_data = train_data.to_numpy()
    label_data = label_data.to_numpy()
# 모든 데이터가 병합된 데이터프레임 확인
    res = encode(train_data,label_data)
    with open('./Final_data_token_SMILE/CLS/'+tox_names[index],'wb') as file:
        pickle.dump(res,file)

353 353


100%|██████████| 6696/6696 [00:00<00:00, 117117.22it/s]
[01:24:09] Explicit valence for atom # 1 N, 4, is greater than permitted
[01:24:09] Explicit valence for atom # 6 N, 4, is greater than permitted
[01:24:09] Explicit valence for atom # 6 N, 4, is greater than permitted
[01:24:09] Explicit valence for atom # 11 N, 4, is greater than permitted
[01:24:09] Explicit valence for atom # 12 N, 4, is greater than permitted
[01:24:09] Explicit valence for atom # 5 N, 4, is greater than permitted
[01:24:09] Explicit valence for atom # 5 N, 4, is greater than permitted
[01:24:09] Explicit valence for atom # 5 N, 4, is greater than permitted
[01:24:09] Explicit valence for atom # 5 N, 4, is greater than permitted
[01:24:09] Explicit valence for atom # 5 N, 4, is greater than permitted
[01:24:09] Explicit valence for atom # 5 N, 4, is greater than permitted


2039 2039


100%|██████████| 40197/40197 [00:00<00:00, 124016.13it/s]


1427 1427


...

100%|██████████| 40204/40204 [00:00<00:00, 122165.91it/s]
[01:25:20] SMILES Parse Error: syntax error while parsing: c1ccc2c(c1)[i+]c3ccccc23
[01:25:20] SMILES Parse Error: Failed parsing SMILES 'c1ccc2c(c1)[i+]c3ccccc23' for input: 'c1ccc2c(c1)[i+]c3ccccc23'


8069 8069


100%|██████████| 161019/161019 [00:01<00:00, 105725.46it/s]


3219 3219


100%|██████████| 64380/64380 [00:00<00:00, 95161.66it/s]


In [12]:
import pandas as pd
import glob
import pickle
# CSV 파일이 있는 폴더 경로 설정
folder_path = './Final_data/REG/'

# 폴더 내의 모든 CSV 파일 경로 가져오기
csv_files = glob.glob(folder_path + '*.csv')
tox_names = []
for file in csv_files:
    name = file.split('/')[3]
    tox_names.append(name[:-4])

# 모든 CSV 파일을 담을 빈 데이터프레임 생성
all_data = pd.DataFrame()
train_Index = ['Smiles','smiles','SMILES']
label_Index = ['Property','expt','exp','p_np']
# 각 CSV 파일을 읽어와서 데이터프레임에 추가
for index,file_path in enumerate(csv_files):
    train_data = []
    label_data = []
    df = pd.read_csv(file_path)
    for t in train_Index:
        try:
            train_data=df[t]
        except:
            pass
    for l in label_Index:
        try:
            label_data = df[l]
        except:
            pass
    train_data = train_data.to_numpy()
    label_data = label_data.to_numpy()
# 모든 데이터가 병합된 데이터프레임 확인
    res = encode(train_data,label_data)
    with open('./Final_data_token_SMILE/REG/'+tox_names[index],'wb') as file:
        pickle.dump(res,file)

642 642


100%|██████████| 9523/9523 [00:00<00:00, 210525.42it/s]


4200 4200


100%|██████████| 83975/83975 [00:00<00:00, 96571.96it/s] 
